In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
path = '../output_data/P3o2n.h5'
with pd.HDFStore(path) as store:
    print(store.info())
    data = store.select(key='dat_0')
data.index = data.index.droplevel(['tau', 'xi', 'kappa_c', 'test', 'approximate'])
data.reset_index()
data['Y'] = data['Y_c'] + data['Y_d']
data_mean = data.groupby(['phi', 'tstep']).mean()
data_std = data.groupby(['phi', 'tstep']).std()

In [ ]:
index_dict = {key: list(value) for key, value in zip(data_mean.index.names, data_mean.index.levels)}
print(data.columns.values)
data.index.names

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2)
axes = [ax for l in axes for ax in l]
fig.set_figwidth(9)
fig.set_figheight(7)

# observables=['local clustering coefficient', 'mean shortest path', 'N_c over N', '[cd] over M']
observables=['N_c over N', '[cd] over M', 'r_c', 'Y_c']
y_axis_labels = ['$r_c$']

def plot_data(ax, mean_data, std_data, observable, l_obs=None, colors=None):
    # select data
    tmp_m = mean_data[[observable]].unstack('phi')
    tmp_s = std_data[[observable]].unstack('phi')
    # adjust column index
    tmp_m.columns = tmp_m.columns.droplevel(0)
    tmp_s.columns = tmp_s.columns.droplevel(0)
    # save column values and reset with human readable names
#     phi_values = tmp_m.columns.values
    phi_values = [0., .5, .75, .85, .9, .95]
    print(phi_values)
#     tmp_m.columns = [f'$\\varphi = {phi}$' for phi in tmp_m.columns]
    # plot data for individual phi values
    if colors is None:
        import matplotlib.colors as mcolors
        colors = [c for _, c in mcolors.TABLEAU_COLORS.items()]
        
    for i, phi in enumerate(phi_values):
        color = colors[i]
        tmp_m_phi = tmp_m[[phi]].values.T[0]
        tmp_s_phi = tmp_s[[phi]].values.T[0]
        t = tmp_m.index.values
        ax.plot(t, tmp_m_phi, color, label=f'$\\varphi = {phi}$')
#         ax.plot(t, tmp_m_phi - tmp_s_phi, color, alpha=0.2)
#         ax.plot(t, tmp_m_phi + tmp_s_phi, color, alpha=0.2)
#         ax.fill_between(t, tmp_m_phi - tmp_s_phi, tmp_m_phi + tmp_s_phi, color=color, alpha=0.2)
    if observable == l_obs:
        ax.legend()
    
    ax.set_ylabel(observable)
    ax.set_xlabel('t')

for i, observable in enumerate(observables):
    plot_data(axes[i], data_mean, data_std, observable, l_obs='Y_c')
    axes[i].set_xlim([300, 800])

fig.tight_layout()
fig.savefig('varphi_scan.pdf')

In [ ]:
data.head()

In [ ]:
nops = [10, 190]
ops = np.zeros(sum(nops), dtype='int')
ops[:nops[0]] = 1
ops